In [1]:
import numpy as np
import numba as nb
from numba import cuda, jit
import time
import math
from Maxpool import Maxpool, forward_paralle, forward_sequential, forward_sequence_pure
from Convolution import Convolution, forward, forward_seq, CNN_forward
from size import Shape
           

C:\Users\nagin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


Layer Convolution
Time seq: 	2.4659945964813232
Time seq pure: 	4.693268299102783
Time jit decorate: 	0.8702709674835205
time cuda.jit decorate: 	1.7392609119415283
Error output - output_paral:	0.0
Error output - out_seq:	0.0
Error output - output_paral_jit:	0.0
(64, 4096)


C:\Users\nagin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [2]:
import pandas as pd
time_run = pd.DataFrame(columns=['seq_pure','seq_lib', 'jit', 'cuda.jit'])
error = pd.DataFrame(columns=['seq_pure','seq_lib', 'jit', 'cuda.jit'])

## Maxpool 

In [3]:
h = 32
w = 128
a = np.random.randint(0, 256 ,(64, w * h)) 
a_size = Shape(h, w)

In [4]:
maxpool = Maxpool(64,Shape(2,2))
s_start = time.time()
output_seq, shape_out = maxpool.forward_sequential(a, a_size)
s_end = time.time()
print(f'{s_end - s_start}')

0.006002902984619141


In [5]:
time_run.loc['maxpool','seq_lib'] = s_end - s_start
error.loc['maxpool','seq_lib'] = "-"

In [6]:
block_size = (16, 16)
# print(f'({shape_out.h // block_size[0] + 1}, {shape_out.w // block_size[1] + 1})')
# grid_h, grid_w = shape_out.h // block_size[0] + 1,shape_out.w // block_size[1] + 1
# grid_size = (grid_h, grid_w)
grid_size = (math.ceil(shape_out[0] / block_size[0]),
             math.ceil(shape_out[1] / block_size[1]))
# print(grid_size)
d_a = cuda.to_device(a)

In [7]:
output_jit = np.empty((maxpool.numKernel, shape_out[0]*shape_out[1]), dtype=float)
start = time.time()
forward_paralle[grid_size, block_size](d_a, (h, w), output_jit, (shape_out[0], shape_out[1]), (2,2), 1)
end = time.time()
print(f'{end - start}')

C:\Users\nagin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


0.2782268524169922


C:\Users\nagin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [8]:
time_run.loc['maxpool','jit'] = end - start
error.loc['maxpool','jit'] = np.sum(abs(output_jit - output_seq))

In [9]:
out_pure = np.empty((maxpool.numKernel, shape_out[0]*shape_out[1]), dtype=float)
start = time.time()
forward_sequence_pure(a, (h, w), out_pure, (shape_out[0], shape_out[1]), (2,2), 1)
end = time.time()
print(f'{end - start}')

0.003002643585205078


In [10]:
time_run.loc['maxpool','seq_pure'] = end - start
error.loc['maxpool','seq_pure'] = np.sum(abs(out_pure - output_seq))

In [11]:

output_paral = np.empty((maxpool.numKernel, shape_out[0]*shape_out[1]), dtype=float)
start = time.time()
forward_paralle[grid_size, block_size](d_a, (h, w), output_paral, (shape_out[0], shape_out[1]), (2,2), 64)
end = time.time()
print(f'{end - start}')

0.002001523971557617


In [12]:
time_run.loc['maxpool', 'cuda.jit'] = end - start
error.loc['maxpool', 'cuda.jit'] = np.sum(abs(output_paral- output_seq))

In [13]:
time_run

,seq_pure,seq_lib,jit,cuda.jit
maxpool,0.003003,0.006003,0.278227,0.002002


In [14]:
error

,seq_pure,seq_lib,jit,cuda.jit
maxpool,0.0,-,0.0,0.0


## Convolution 

In [15]:
sizeKernel = Shape(3,3)
cnn = Convolution(64,sizeKernel)
aAddPaddinf = cnn.addPadding(a[:1], (h, w), 1)
w_add = w + cnn.padding*2
h_add = h + cnn.padding*2

In [16]:
s_start = time.time()
output = cnn.forward(aAddPaddinf, (h,w), 1, "relu")
s_end = time.time()
time_run.loc['Convolution','seq_lib'] = s_end - s_start
error.loc['Convolution','seq_lib'] = '-'

In [17]:
out_seq = np.empty((cnn.numKernel, h*w), dtype=float)

seq_start = time.time()
#CNN_forward(input, in_shape, output, out_shape, num_cnn active, size_kernel, weight, bias):
forward_seq(aAddPaddinf.tolist(), (h_add, w_add),1, out_seq, (h, w), 64,(3,3), cnn.weight, cnn.bias)
seq_end = time.time()
time_run.loc['Convolution','seq_pure'] = seq_end - seq_start
error.loc['Convolution','seq_pure'] = np.sum(abs(output - out_seq))

In [18]:
output_paral_jit = np.empty((cnn.numKernel, h*w), dtype=float)
jit_p_start = time.time()
#CNN_forward(input, in_shape, output, out_shape, num_cnn active, size_kernel, weight, bias):
forward(aAddPaddinf, (h_add, w_add), 1,output_paral_jit, (h, w), 64, (3,3), cnn.weight, cnn.bias)
jit_p_end = time.time()
time_run.loc['Convolution','jit'] = jit_p_end - jit_p_start
error.loc['Convolution','jit'] = np.sum(abs(output - output_paral_jit))

In [19]:
output_paral = np.empty((cnn.numKernel, h*w), dtype=float)
block_size = (16, 16)
# print(f'({shape_out.h // block_size[0] + 1}, {shape_out.w // block_size[1] + 1})')
# grid_h, grid_w = shape_out.h // block_size[0] + 1,shape_out.w // block_size[1] + 1
# grid_size = (grid_h, grid_w)
grid_size = (math.ceil(h / block_size[0]),
             math.ceil(w / block_size[1]))
# print(grid_size)
p_start = time.time()
#CNN_forward(input, in_shape, output, out_shape, num_cnn active, size_kernel, weight, bias):
CNN_forward[grid_size, block_size](aAddPaddinf, (h_add, w_add), 1, output_paral, (h, w), 64 ,(3,3), cnn.weight, cnn.bias)
p_end = time.time()
time_run.loc['Convolution','cuda.jit'] = p_end - p_start
error.loc['Convolution','cuda.jit'] = np.sum(abs(output - output_paral))

C:\Users\nagin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
C:\Users\nagin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [20]:
error

,seq_pure,seq_lib,jit,cuda.jit
maxpool,0.0,-,0.0,0.0
Convolution,0.0,-,0.0,0.0


In [21]:
time_run

,seq_pure,seq_lib,jit,cuda.jit
maxpool,0.003003,0.006003,0.278227,0.002002
Convolution,6.111412,2.462196,0.004997,0.009995


## CNN

Cấu trúc mô hình: 

In [22]:
a = np.random.randint(0, 256 ,(w * h)) 
w = 128
h = 32

In [23]:
con_1 = Convolution(64, Shape(3,3))
maxpool_1 = Maxpool(64, Shape(2,2))
con_2 = Convolution(128,Shape(3,3))
maxpool_2 = Maxpool(128, Shape(2,2))
con_3 = Convolution(256,Shape(3,3))
con_4 = Convolution(256,Shape(3,3))
maxpool_3 = Maxpool(256, Shape(2,2))
con_5 = Convolution(512,Shape(3,3))

input = a.reshape((1,w*h))

start = time.time()
out_conv1 = con_1.forward(con_1.addPadding(input, (32,128), 1), (32,128), 1, 'relu')
out_pooling1, out_shape = maxpool_1.forward_sequential(out_conv1,Shape(32,128))
out_conv2 = con_2.forward(con_2.addPadding(out_pooling1, out_shape, 64), (16,64), 64, 'relu')
out_pooling2, out_shape = maxpool_2.forward_sequential(out_conv2,Shape(16,64))
out_conv3 = con_3.forward(con_3.addPadding(out_pooling2,out_shape,128), out_shape, maxpool_2.numKernel, 'relu')
out_conv4 = con_4.forward(con_4.addPadding(out_conv3, out_shape, 256),(8,32), 256,'relu')
out_pooling3, out_shape = maxpool_3.forward_sequential(out_conv4,Shape(8,32))
out_conv5 = con_5.forward(con_5.addPadding(out_pooling3,out_shape, maxpool_3.numKernel), (4,16), 256,'relu')
end = time.time()
time_run.loc["CNN",'seq_lib'] = (end-start)

In [24]:
block_size = (16, 16)
grid_size = (math.ceil(h / block_size[0]),
            math.ceil(w / block_size[1]))
p_out_conv1 = np.empty((con_1.numKernel, 32*128),dtype=float)
p_out_conv2 = np.empty((con_2.numKernel, 16*64),dtype=float)
p_out_conv3 = np.empty((con_3.numKernel, 8*32),dtype=float)
p_out_conv4 = np.empty((con_4.numKernel, 8*32),dtype=float)
p_out_conv5 = np.empty((con_5.numKernel, 4*16),dtype=float)
p_maxpool_1 = np.empty((con_1.numKernel, 16*64),dtype=float)
p_maxpool_2 = np.empty((con_2.numKernel, 8*32),dtype=float)
p_maxpool_3 = np.empty((con_4.numKernel, 4*16),dtype=float)

para_s_time = time.time()
CNN_forward[grid_size, block_size](cuda.to_device(con_1.addPadding(input,(32,128),1)),(32 + con_1.padding*2, 128+ con_1.padding*2), 1, 
                                                p_out_conv1, (32, 128), 64 ,(3,3), con_1.weight, con_1.bias)

forward_paralle[grid_size, block_size](cuda.to_device(p_out_conv1), (32, 128), p_maxpool_1, (16, 64), (2,2), 64)

CNN_forward[grid_size, block_size](cuda.to_device(con_2.addPadding(p_maxpool_1,(16, 64),64)),(16 + con_2.padding*2, 64+ con_2.padding*2), 64, 
                                                p_out_conv2, (16, 64), 128 ,(3,3), con_2.weight, con_2.bias)

forward_paralle[grid_size, block_size](cuda.to_device(p_out_conv2), (16, 64), p_maxpool_2, (8, 32), (2,2), 128)

CNN_forward[grid_size, block_size](cuda.to_device(con_3.addPadding(p_maxpool_2,(8, 32),128)),(8 + con_3.padding*2, 32+ con_3.padding*2), 128, 
                                                p_out_conv3, (8, 32), 256 ,(3,3), con_3.weight, con_3.bias)

CNN_forward[grid_size, block_size](cuda.to_device(con_4.addPadding(p_out_conv3,(8, 32),256)),(8 + con_4.padding*2, 32+ con_4.padding*2), 256, 
                                                p_out_conv4, (8, 32), 256 ,(3,3), con_4.weight, con_4.bias)

forward_paralle[grid_size, block_size](cuda.to_device(p_out_conv4), (8, 32), p_maxpool_3, (4, 16), (2,2), 256)

CNN_forward[grid_size, block_size](cuda.to_device(con_5.addPadding(p_maxpool_3,(4, 16),256)),(4 + con_5.padding*2, 16+ con_5.padding*2), 256, 
                                                p_out_conv5, (4, 16), 512 ,(3,3), con_5.weight, con_5.bias)
para_e_time = time.time()
time_run.loc["CNN",'cuda.jit'] = (para_e_time - para_s_time)

C:\Users\nagin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
C:\Users\nagin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cuda\dispatcher.py:536: NumbaPerformanceWarning: Grid size 16 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
C:\Users\nagin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\numba\cuda\cudadrv\devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


In [25]:
assert np.mean(np.abs(p_out_conv1 - out_conv1)) == 0.000
assert np.mean(np.abs(p_maxpool_1 - out_pooling1)) == 0.000
assert np.mean(np.abs(p_out_conv2 - out_conv2)) == 0.000
assert np.mean(np.abs(p_maxpool_2 - out_pooling2)) == 0.00
assert np.mean(np.abs(p_out_conv3 - out_conv3)) == 0.00
assert np.mean(np.abs(p_out_conv4 - out_conv4)) == 0.00
assert np.mean(np.abs(p_maxpool_2 - out_pooling2))==0.00
assert np.mean(np.abs(p_out_conv5 - out_conv5))==0.00

In [26]:
time_run

,seq_pure,seq_lib,jit,cuda.jit
maxpool,0.003003,0.006003,0.278227,0.002002
Convolution,6.111412,2.462196,0.004997,0.009995
CNN,NaN,319.406635,NaN,0.973556


In [27]:
error

,seq_pure,seq_lib,jit,cuda.jit
maxpool,0.0,-,0.0,0.0
Convolution,0.0,-,0.0,0.0
